- Table of contents
- [Todo App](#todo-app)
  - [Create prompt](#create-prompt)

# Todo App

- It can do following things on the behalf of user:
  - Create Todo
  - Update Todo
  - Delete Todo
  - Get Todo

## Create Prompt

- Here I am going to give context how they behave on user input.
- I use [**`few_shot_prompting`**](../../05_few_shot_prompting/few_shot_prompting.md), that them to learn from examples.
- I will give them few examples of how to behave on user input.

In [156]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage, ToolMessage

system_message = """
You are a helpful assistant for todo management (Create, Update, Delete, Get).
1.  **Analyze Intent:** Determine the user's primary action: Create, Update, Delete, or Get.
2.  **Specify Todos:** For Update/Delete operations, ensure the correct todo is targeted; ask to clarify if needed (e.g., if multiple match or none are found). For Get, determine if all todos or a filtered list is needed.
3.  **Batch Operations:** Recognize and process requests involving multiple todos.
4.  **Action & Feedback:** Execute the request and confirm completion or explain issues.
5. **Choose correct tool:** Determine the correct tool to call such as Create, Update, Delete or Get.
"""
examples = [
    HumanMessage(
        content="Create a new todo item for grocery shopping. I need to buy milk and eggs. it is due tomorrow. and have a priority of 1.",
    ),
    AIMessage(
        content="Creating a new todo item for grocery shopping with the following details:\n- Task: Buy milk and eggs\n- Due Date: Tomorrow\n- Priority: 1\n",
    ),
    AIMessage(
        "",
        name="example_assistant",
        tool_calls=[
            {
                "name": "create_todo",
                "args": {
                    "name": "Grocery shopping",
                    "description": "Buy milk and eggs",
                    "due_date": "tomorrow",
                    "priority": 1
                },
                "id": "1"
            }
        ]
    ),
    ToolMessage("", tool_call_id="1"),
    HumanMessage(
        content="Update the grocery shopping todo item to include bread and change the due date to next week.",
    ),
    AIMessage(
        content="Updating the grocery shopping todo item with the following details:\n- Additional Task: Buy bread\n- New Due Date: Next week\n",
    ),
    AIMessage(
        "",
        name="example_assistant",
        tool_calls=[
            {
                "name": "update_todo",
                "args": {
                    "id": "1",
                    "description": "Buy milk, eggs, and bread",
                    "due_date": "next week"
                },
                "id": "2"
            }
        ]
    ),
    ToolMessage("", tool_call_id="2"),
    HumanMessage(
        content="Can you add a new todo item for laundry? It should be due next Monday. and also add going to the gym.",
    ),
    AIMessage(
        content="Adding a new todo item for laundry with the following details:\n- Task: Laundry\n- Due Date: Next Monday\n",
    ),
    AIMessage(
        "",
        name="example_assistant",
        tool_calls=[
            {
                "name": "create_todo",
                "args": {
                    "name": "Laundry",
                    "description": "",
                    "due_date": "next Monday",
                    "priority": 1
                },
                "id": "3"
            },
            {
                "name": "create_todo",
                "args": {
                    "name": "Going to the gym",
                    "description": "",
                    "due_date": "",
                    "priority": 1
                },
                "id": "4"
            }
        ]
    ),
    ToolMessage("", tool_call_id="3"),
    ToolMessage("", tool_call_id="4"),
]

prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content=system_message),
        *examples,
        ("human", "{input}")
    ]
)

## Creating Output Structure

### Create tool arguments Structure

In [ ]:
from typing import List, Optional, Union
from pydantic import BaseModel, Field

# Todo Schema


class TodoSchema(BaseModel):
    id: str = Field(description="Ids of the todo item.")
    name: str = Field(description="Name of the todo.")
    description: Optional[str] = Field(
        description="Description of the todo.", default=None)
    due_date: Optional[str] = Field(
        description="Due data of todo.", default=None)
    priority: Optional[int] = Field(
        description="Priority of todo.", default=None)


# Create todo schema
class CreateTodoSchema(BaseModel):
    name: str = Field(description="Name of the todo")
    description: Optional[str] = Field(description="Description of the todo")
    due_date: Optional[str] = Field(description="Due data of todo")
    priority: Optional[int] = Field(description="Priority of todo")


# Update todo schema
class UpdateTodoSchema(BaseModel):
    id: str = Field(description="Id of the todo that we want to update.")
    name: Optional[str] = Field(description="Update name of the todo")
    description: Optional[str] = Field(
        description="Updated description of the todo")
    due_date: Optional[str] = Field(description="updated due date of the todo")
    priority: Optional[int] = Field(description="Updated priority of the todo")


# Delete todo schema
class DeleteTodoSchema(BaseModel):
    id: Union[str, List[str]] = Field(
        description="Id of the todo that we want to delete.")

# Get todo schema


class GetTodoSchema(BaseModel):
    id: Optional[Union[str, List[str]]] = Field(
        description="Id of the todo what we specially wants.")

<function pydantic.main.BaseModel.json(self, *, include: 'IncEx | None' = None, exclude: 'IncEx | None' = None, by_alias: 'bool' = False, exclude_unset: 'bool' = False, exclude_defaults: 'bool' = False, exclude_none: 'bool' = False, encoder: 'Callable[[Any], Any] | None' = PydanticUndefined, models_as_dict: 'bool' = PydanticUndefined, **dumps_kwargs: 'Any') -> 'str'>

## Creating Tools

In [161]:
from uuid import uuid4
from langchain_core.tools import tool


todos_data: List[TodoSchema] = []


@tool(args_schema=CreateTodoSchema)
def create_todo(name: str, **kwargs):
    """
    Create a new todo item.

    Args:
        name (str): The name of the todo item.
        **kwargs: Additional attributes for the todo item.
    """
    todo = {
        "id": str(uuid4()),
        "name": name,
        **kwargs
    }
    todos_data.append(TodoSchema(**todo))
    return todo


@tool(args_schema=UpdateTodoSchema)
def update_todo(id: str, **kwargs):
    """
    Update an existing todo item.

    Args:
        id (str): The ID of the todo item to update.
        **kwargs: Updated attributes for the todo item.
    """
    try:
        for index, todo in enumerate(todos_data):
            if todo.id == id:
                updated_value = todo.model_copy(update=kwargs)
                todos_data[index] = updated_value
    except Exception as e:
        return {"error": str(e)}


@tool(args_schema=DeleteTodoSchema)
def delete_todo(id: Union[str, List[str]]):
    """
    Delete an existing todo item.

    Args:
        id (Union[str, List[str]]): The ID(s) of the todo item(s) to delete.
    """
    global todos_data
    ids = [id] if isinstance(id, str) else id
    todos_data = [todo for todo in todos_data if id in ids]


@tool(args_schema=GetTodoSchema)
def get_todo(id: Optional[Union[str, List[str]]]):
    """
    Get an existing todo item.

    Args:
        id (Optional[Union[str, List[str]]]): The ID(s) of the todo item(s) to get.
    """
    if id is None:
        return [todo.model_dump for todo in todos_data]

    if isinstance(id, str):
        return next((todo for todo in todos_data if todo.id == id), None)
    else:
        return [todo.model_dump() for todo in todos_data if todo.id in id]


tools = [create_todo, update_todo, delete_todo, get_todo]
tools_with_name = {
    tool.name: tool for tool in tools}

## Initialize Model

In [153]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-nano", temperature=0)
llm_with_tools = llm.bind_tools(tools)

## Create Chains 

In [ ]:
from langchain_core.runnables import chain, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

# Chat History
chat_history = []


def tool_calling(ai_message: AIMessage):
    tool_calls = []

    if len(ai_message.tool_calls) != 0:
        for tool in ai_message.tool_calls:
            selected_tool = tools_with_name[tool["name"]]
            tool_response = selected_tool.invoke(tool)

            tool_calls.append(tool_response)

    return tool_calls


def get_model_response(tool_message):
    return llm.invoke(tool_message)


@chain
def chain_executor(input):
    chain = prompt | llm_with_tools | RunnableLambda(
        tool_calling) | RunnableLambda(get_model_response) | StrOutputParser()

    return chain.invoke(input)


chain_executor.invoke(
    "Can you add a new todo item for going luandry tomorrow")

BadRequestError: Error code: 400 - {'error': {'message': "Invalid parameter: messages with role 'tool' must be a response to a preceeding message with 'tool_calls'.", 'type': 'invalid_request_error', 'param': 'messages.[0].role', 'code': None}}

## Main Execution Function

In [ ]:
def main():
    while True:
        user_input = input("Specify your todo items.")
        if len(user_input) == 0:
            print("Error: Invalid request.")

        llm_response = chain_executor.invoke(user_input)
        print(llm_response)

        choice = input("Do you want to add more todos. (y/n). ")
        if choice == "y" or choice == "yes":
            break

## Pydantic Tutorial

In [ ]:
from pydantic import BaseModel, Field, field_validator
import re


class Profile(BaseModel):
    name: str = Field(description="Name of the user")
    email: str = Field(description="Email of the user")
    profile_image: Optional[str] = Field(
        description="Profile Image of the user", default=None)
    phone_number: Optional[str] = Field(
        description="Phone Number of the user", default=None)

    # Custom validation for email
    @field_validator("email")
    def validate_email(cls, v):
        if not re.match(r"[^@]+@[^@]+\.[^@]+", v):
            raise ValueError("Invalid email format")
        return v


data_list = []


try:
    data_list.append(
        Profile(
            name="Abdur Rab Khan",
            email="abdurrabkhan222@gmail.com"
        )
    )

    data_list.append(
        Profile(
            name="Raheem Khan",
            email="raheemkhan@gmail.com",
        )
    )

    data_list.append(
        Profile(
            name="Rahul Khan",
            email="rahul@gmail.com"
        )
    )

    for i, data in enumerate(data_list):
        if data.email == "abdurrabkhan222@gmail.com":
            data_list[i] = data.model_copy(
                update={"email": "abdurrabkhan3263@gmail.com"})
except ValueError as e:
    print(f"Error: {e}")

for data in data_list:
    print(data.model_dump())


num_list = [1, 2, 3, 4, 5, 6]
sixth_iterator = (n for n in num_list if n == 6)

print(sixth_iterator)
print(next(sixth_iterator, None))

{'name': 'Abdur Rab Khan', 'email': 'abdurrabkhan3263@gmail.com', 'profile_image': None, 'phone_number': None}
{'name': 'Raheem Khan', 'email': 'raheemkhan@gmail.com', 'profile_image': None, 'phone_number': None}
{'name': 'Rahul Khan', 'email': 'rahul@gmail.com', 'profile_image': None, 'phone_number': None}
<generator object <genexpr> at 0x000001C0C80C8FB0>
6
